<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Introduction and Basic Querying
</p>
</div>

Data Science Cohort Live NYC MAY 2022
<p>Phase 1: Topic 6</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    

#### Databases

Data for organizations: typically stored in a **database**.

- Database: organized collection of structured information.
- Typically fairly large and hosted by dedicated servers.
- A database controlled by a database management system (DBMS).



<div align = "center">
    <center><img src="Images/dbms.jpg" align = "center" width="500"/></center>
</div>

With DBMS: store, update, share, and access specific subset of database. 

- CSVs and JSONs are often extracts of some data from a database.

#### The relational database
<br>
<div align = "center">
    <center><img src="Images/tables_relational.png" align = "center" width="700"/></center>
</div>
  

Tables are linked together by relationships.

  Why relational databases are great and what came before (first 5 modules): <br>
  https://mariadb.com/kb/en/database-theory/

Common DBMS for Relational Databases
<br>
<div>
    <center><img src="Images/postgresql.png"  width="200"/></center>
    <center><img src="Images/mysql.png"  width="300"/></center>
    <center><img src="Images/SQLite-Python.jpg" align = "center" width="200"/></center>
</div>
      
</div>

#### Structured Query Language

- De-facto king of languages for interacting with relational databases and their DBMS.
- Invaluable for you in almost any data role. 
- You will use it often to get the data that you need for your analyses.

Let's get into using SQL!


In [ ]:
# import sqlite
import sqlite3
import pandas as pd

Interacting with a sqlite database.
- Connect to database 
- Create interface to ask database for stuff
- After connection established:
    1. Execute query 
    2. Fetch result 
- Close connection to database when done with queries.


Commands:
- conn = sqlite3.connect(database)
- cursor_obj = connection.cursor()
- Query database/get result:
    1. cursor.execute(SQL_query)
    2. cursor.fetchall()
- connection.close()

This opens a connection to the flights database and creates cursor object.

In [ ]:
con = sqlite3.connect('data/flights.db')
cursor_obj = con.cursor()

See tables in the sqlite database via command line.

In [ ]:
%%bash 

sqlite3 data/flights.db
.tables

Get schema of sqlite database: whole or single table

In [ ]:
%%bash
sqlite3 data/flights.db
.schema airports

#### SELECT statement

Let's explore the following clauses and structure:

```sql
SELECT columns ( * for all fields)
FROM table
WHERE condition
ORDER BY columns
LIMIT number (limits to first n entries of table)
```

Execute SQL command:

In [ ]:
query = cursor_obj.execute("""
SELECT *
FROM airports
LIMIT 1
""")

In [ ]:
query.fetchall()

- execute, fetchall protocol gets data in python list form.
- can issue commands to create things in database.

When *just retrieving* tabular data:

- Pandas has pd.read_sql(query, database_connection) command.

In [ ]:
airports_query = \
"""
SELECT *
FROM airports
"""

In [ ]:
query_result_df = pd.read_sql(airports_query, con)
query_result_df.head()

Selecting particular columns:

In [ ]:
query_result_df = pd.read_sql("""

SELECT city, country 
FROM airports

""", con)

query_result_df.head(5)

#### DISTINCT
- Select only unique results of query.

In [ ]:
query_result = pd.read_sql("""

SELECT DISTINCT country
FROM airports

""", con)

query_result.head()

#### AS modifier

`AS`: renaming a column in the query and returned result.

In [ ]:
sql_query = pd.read_sql("""

SELECT city AS "Airport City", 
       country AS "Airport Country"
FROM airports

""", con)

sql_query.head()

#### Functions
Can use predefined functions in SELECT statements:
<br>
<br>
SELECT FUNCTION(columns)
<br>
FROM table

Applies function to the each column and spits out new table.

Functions can be: 
- Transformations on columns:
    - Convert each element in a string column to upper case.
- Aggregation:
    - statistic on each field in a table 
    - e.g. max, min, mean, sum, etc.

#### Transformation

In [ ]:
sql_query_df = pd.read_sql("""

SELECT UPPER(name) AS "AIRPORT NAMES IN CAPS",
       LENGTH(name) AS "Airport Name Length"
       
FROM airports

""", con)

sql_query_df.head()

#### Aggregation
- MAX()
- MIN()
- SUM()
- AVG()
- COUNT()

Get count of entries in table:

In [ ]:
sql_query_df = pd.read_sql("""

SELECT COUNT(*) AS count
FROM airports


""", con)

sql_query_df

Get count of non-missing entries in a given field:


SELECT COUNT(field_name)
FROM table

In [ ]:
sql_query_df = pd.read_sql("""

SELECT COUNT(code) as code_count
FROM airports



""", con)

sql_query_df

Distinct countries in country field of airports table:

In [ ]:
sql_query_df = pd.read_sql("""

SELECT COUNT(DISTINCT country) as country_count
FROM airports


""", con)

sql_query_df

Let's take a look at our original table again.

In [ ]:
sql_query_df = pd.read_sql("""

SELECT * 
FROM airports
LIMIT 5

""", con)

sql_query_df

Get maximum altitude in dataset along with corresponding airport. 

In [ ]:
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
    MAX(altitude) AS "Altitude (ft)"      
FROM airports

""", con)

sql_query_df

Something is wrong here...

In [ ]:
%%bash
sqlite3 data/flights.db
.schema airports

#### `CAST()`

- can enforce typing on a given field.

In [ ]:
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
       MAX(CAST(altitude as float)) AS "Altitude (ft)"
       
FROM airports

""", con)

sql_query_df

Let's get the average altitude:

In [ ]:
sql_query_df = pd.read_sql("""

SELECT AVG(CAST(altitude as int)) AS "Average Altitude (ft)"
FROM airports

""", con)

sql_query_df

#### Exercise


Which country has the northern-most airport? 

> _Hint: Look for the highest latitude_

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""
            
SELECT name AS airport_name, country, MAX(CAST(latitude as int))
FROM airports


""", con)

```
</details>

So far:
- Selecting fields (columns) and computing functions of them.

Want to: 
- Select records (rows) matching particular criteria.

#### WHERE: Selecting rows via filtering

SELECT columns
<br>
FROM table
<br>
WHERE filter_column meets condition

#### Filtering by comparison: WHERE statement

Commonly filter by how column compares to a value.

SELECT column1, column2 
FROM table
WHERE column2 > number

| Operator | Meaning | 
| --- | --- |
| \= | equal |
| \<> | not equal |
| \<  |  less than |
| \> | greater than |
| \<=  | less than or equal to |
| \>= | greater than or equal to |


In [ ]:
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(latitude AS int) AS "Airport Latitude",
       CAST(altitude AS int) AS "Altitude (ft)"
       
FROM airports

WHERE "Altitude (ft)" >= 12500

""", con)

sql_query_df

#### NOT operator
- Before conditional in `WHERE` clause.
- Negates conditional.

In [ ]:
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(latitude AS int) AS "Airport Latitude",
       CAST(altitude AS int) AS "Altitude (ft)"
       
FROM airports

WHERE NOT "Altitude (ft)" <= 12500

""", con)

sql_query_df

#### Chaining conditionals for filtering
- AND
- OR


#### AND
Get all airports with:
- Altitude > 12500 ft.
- Latitude > 30 degrees

In [ ]:
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(latitude AS int) AS "Airport Latitude",
       CAST(altitude AS int) AS "Altitude (ft)"
       
FROM airports

WHERE "Altitude (ft)" > 12500 
AND 
"Airport Latitude" > 30

""", con)

sql_query_df

#### OR
Get all airports and their latitude/longitude in Iceland and Denmark.

In [ ]:
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
        country,
       CAST(latitude AS int) AS "Latitude",
       CAST(longitude AS int) AS "Longitude"
       
FROM airports

WHERE country = "Iceland" OR country = "Denmark"

""", con)

sql_query_df.head(12)

#### Exercise

Get the name, lat/long, and altitude for the airport with the maximum altitude considering only airports in Chile and Argentina.

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
sql_query_df = pd.read_sql("""

SELECT name AS airport_name,
    CAST(latitude as float) AS latitude,
    CAST(longitude as float) AS longitude,
    MAX(CAST(altitude as int)) AS altitude
    
    FROM airports
    
    WHERE country = "Chile" OR country = "Argentina"
    

""", con)

sql_query_df

```
</details>

#### BETWEEN

- Writing multiple inequalities for selecting ranges with `WHERE` statement: annoying.

- BETWEEN ... AND ... solves this problem.


E.g. Get the airport name, country, latitude, and longitude where latitude is between 48 and 51.

In [ ]:
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
        country,
       CAST(latitude AS int) AS "Latitude",
       CAST(longitude AS int) AS "Longitude"
       
FROM airports

WHERE Latitude BETWEEN 48 AND 51
""", con)

sql_query_df


<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python
sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
        country,
       CAST(latitude AS int) AS "Latitude",
       CAST(longitude AS int) AS "Longitude",
       CAST(altitude AS int) AS "Altitude"
       
FROM airports

WHERE Altitude BETWEEN 0 AND 2000
""", con)

sql_query_df
```
</details>

#### IN / NOT IN

- Get all records with a given field taking on a set of values.

SELECT col1, col2 <br> 
FROM table <br>
WHERE col1 IN (col_value1, col_value2, ...) <br>
- E.g.,

    Get name, latitude, longitude for airports in Brazil, Argentina, Chile, and Uruguay 

In [ ]:

sql_query_df = pd.read_sql("""

SELECT name AS "Airport Name", 
        country,
       CAST(latitude AS int) AS "Latitude",
       CAST(longitude AS int) AS "Longitude"
       
FROM airports

WHERE country IN ("Argentina", "Brazil", "Chile", "Uruguay")
""", con)

sql_query_df


##### Filtering rows with null (or no null) values in `WHERE` statement
- IS NULL
- IS NOT NULL

The `IS` operator:
- necessary when working with `NULL` values 
- Cannot use = or <> operators here.  

In [ ]:
pd.read_sql("""

SELECT name AS "Airport Name",
       code AS "Airport Code"
       
FROM airports

WHERE "Airport Code" IS NULL

""", con)

#### Exercise:

Get the count of records with nulls in the code column of the airport table

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT COUNT(*)
FROM airports
WHERE code IS NULL

""", con)
```
</details>

#### ORDER BY: Sorting Results

Use `ORDER BY` to identify the column(s) you want to sort on:
- `ASC` for ascending order (default)
- `DESC` for descending order



Get the name and code of airports with:
- top 10 highest altitudes.
- in order of decreasing altitude. 

Then get lowest 10 in increasing order.

In [ ]:
pd.read_sql("""

SELECT name AS "Airport Name", 
       code,
       CAST(altitude AS int) AS "Altitude (ft)"
       
FROM airports

ORDER BY "Altitude (ft)" DESC

LIMIT 10
         
""", con)

Ordering on multiple columns:

- Can order on one column first and then on another.

E.g., order by:
- decreasing latitudes
- increasing longitudes
- Altitude > 10000 ft.

In [ ]:
pd.read_sql("""

SELECT name AS "Airport Name",
       CAST(altitude AS int) AS "Altitude (ft)",
       CAST(latitude AS int) AS "Airport Latitude",
       CAST(longitude AS int) AS "Airport Longitude"    
FROM airports
WHERE "Altitude (ft)" > 10000
ORDER BY "Airport Latitude" DESC, "Airport Longitude" 

""", con)

#### CASE WHEN STATEMENTS
Use `CASE` to create new columns using conditional logic:

E.g. partition altitudes into high, medium and low.

In [ ]:
sql_query = pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(altitude AS int) AS "Altitude (ft)",
       CASE 
            WHEN CAST(altitude AS int) > 1000 THEN "High"
            WHEN CAST(altitude AS int) < 100 THEN "Low"
            ELSE "Moderate"
       END AS "Altitude Category"
       
FROM airports
         
LIMIT 20

""", con)

sql_query

# Exercises

## Country List


Create a list of countries with airports and order them alphabetically A-Z.

> _Hint: You will need to remove duplicates._

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT DISTINCT country
FROM airports
ORDER BY country ASC

""", con)
```
</details>

## Southern Airports

Get the name, country and latitude of the 10 southern-most airports. 

> _Hint: Look for the smallest latitude._

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT name, country, CAST(latitude AS int) AS "Airport Latitude"
FROM airports
ORDER BY "Airport Latitude" ASC
LIMIT 10

""", con)
``` 
</details>

## Active UK Airlines

Create a list of active airlines in the United Kingdom from the airlines table. 

> _Hint: You will need to explore the airlines table to figure out how to do this._

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT name
FROM airlines
WHERE active='Y'
AND country='United Kingdom'

""", con)
```
</details>